In [1]:
cd '/Users/naman_shivangi/Documents/Data Science/DATASETS/Datasets'

/Users/naman_shivangi/Documents/Data Science/DATASETS/Datasets


In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
import statsmodels.api as sm
import xlsxwriter

In [3]:
data=pd.read_excel('VIFXGBDataset 1.xlsx')

In [4]:
train_data=data[data.Sample=='Train']
train_data=train_data.reset_index()
train_data.drop(['index','Sample'],axis=1,inplace=True)

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DepVar  674 non-null    int64  
 1   x1      674 non-null    float64
 2   x2      674 non-null    float64
 3   x3      674 non-null    int64  
 4   x4      674 non-null    float64
 5   x5      674 non-null    float64
 6   x6      674 non-null    int64  
 7   x7      674 non-null    float64
 8   x8      674 non-null    float64
 9   x9      674 non-null    float64
 10  x10     674 non-null    int64  
 11  x11     674 non-null    float64
 12  x12     674 non-null    float64
 13  x13     674 non-null    float64
 14  x14     674 non-null    int64  
dtypes: float64(10), int64(5)
memory usage: 79.1 KB


In [5]:
test_data=data[data.Sample=='Test']
test_data=test_data.reset_index()
test_data.drop(['index','Sample'],axis=1,inplace=True)

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DepVar  21 non-null     int64  
 1   x1      21 non-null     float64
 2   x2      21 non-null     float64
 3   x3      21 non-null     int64  
 4   x4      21 non-null     float64
 5   x5      21 non-null     float64
 6   x6      21 non-null     int64  
 7   x7      21 non-null     float64
 8   x8      21 non-null     float64
 9   x9      21 non-null     float64
 10  x10     21 non-null     int64  
 11  x11     21 non-null     float64
 12  x12     21 non-null     float64
 13  x13     21 non-null     float64
 14  x14     21 non-null     int64  
dtypes: float64(10), int64(5)
memory usage: 2.6 KB


In [6]:
## Columns for model prep
x_var=list(train_data)
x_var.remove('DepVar')
y_var=['DepVar']

print('X Columns :',x_var)
print('Y Columns :',y_var)

X Columns : ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14']
Y Columns : ['DepVar']


In [7]:
#training model

#train_X and Train_y
X_variable=train_data[x_var]
Y_variable=train_data[y_var]

#adding constant to train_X
X_variable=sm.add_constant(X_variable)
X_variable.head()

variables_dropped_for_pvalue=[]


#Sequential drop of variable with p value>0.05
x=True

while x==True:

    #creating model
    linear_model=sm.OLS(Y_variable,X_variable).fit()
    print('Shape X Var : ',X_variable.shape)
    print('Shape Y var : ',Y_variable.shape)
    
    #sorting p values high to low
    pvaules_high_to_low=linear_model.pvalues.sort_values(ascending=False)
    
    #removing 'constant' variable from list of p values as we cannot drop constant varible while creating model
    pvaules_high_to_low.drop('const',inplace=True)
    
    #converting variable names as column
    pvaules_high_to_low=pvaules_high_to_low.reset_index()
    
    #renaming columns
    pvaules_high_to_low=pvaules_high_to_low.rename(columns={'index':'Variable_Name',0:'pvalue'})
    
    #CAPTURING HIGHEST PVALUE VARIABLE IN DATAFRAME
    highest_pvalue=pd.DataFrame()
    highest_pvalue=pvaules_high_to_low.loc[[0],:]
    
    ##Checking if pvalue is greater than or equal to 0.05 then droppping variable
    if(highest_pvalue.pvalue[0]>=0.05):
        print(highest_pvalue.Variable_Name[0],' has p _values : ',highest_pvalue.pvalue[0],' which is greater than 0.05')
        variables_dropped_for_pvalue.append(highest_pvalue.Variable_Name[0])
        X_variable.drop(highest_pvalue.Variable_Name[0],axis=1,inplace=True)
        
    else:
        print('\n')
        print('Now All variables have p value less than 0.05')
        x=False

print(X_variable.shape)

Shape X Var :  (674, 15)
Shape Y var :  (674, 1)
x6  has p _values :  0.7879049903760391  which is greater than 0.05
Shape X Var :  (674, 14)
Shape Y var :  (674, 1)
x12  has p _values :  0.6811108433428585  which is greater than 0.05
Shape X Var :  (674, 13)
Shape Y var :  (674, 1)
x9  has p _values :  0.714861521082686  which is greater than 0.05
Shape X Var :  (674, 12)
Shape Y var :  (674, 1)
x14  has p _values :  0.46506475044530626  which is greater than 0.05
Shape X Var :  (674, 11)
Shape Y var :  (674, 1)
x11  has p _values :  0.28516008859924297  which is greater than 0.05
Shape X Var :  (674, 10)
Shape Y var :  (674, 1)
x13  has p _values :  0.2226557132617779  which is greater than 0.05
Shape X Var :  (674, 9)
Shape Y var :  (674, 1)
x10  has p _values :  0.21170654847186768  which is greater than 0.05
Shape X Var :  (674, 8)
Shape Y var :  (674, 1)
x2  has p _values :  0.20235172235525256  which is greater than 0.05
Shape X Var :  (674, 7)
Shape Y var :  (674, 1)
x1  has p 

### Train data Scoring

In [8]:
pred_DepVar=linear_model.predict(X_variable)
train_data=pd.concat((train_data,pred_DepVar),axis=1)
train_data=train_data.rename(columns={0:'pred_DepVar'})
train_data['Sample']='Train'
train_data.head()

,DepVar,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,pred_DepVar,Sample
0,15090000,1.92,0.0,0,1.92,16.32,0,0.0,16.32,0.0,0,6.0,6.0,0.0,0,3.945436e+07,Train
1,13405233,0.00,0.0,4,4.00,0.00,2,5.0,7.00,0.0,1,2.0,3.0,0.0,0,4.947422e+07,Train
2,70390700,0.00,0.0,0,0.00,0.00,25,0.0,25.00,0.0,4,0.0,4.0,0.0,4,5.042671e+07,Train
3,80030330,0.00,1.0,0,1.00,0.00,1,0.0,1.00,0.0,1,0.0,1.0,0.0,1,5.042671e+07,Train
4,89201260,0.00,0.0,0,0.00,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0,5.042671e+07,Train


### Test Data Scoring

In [9]:
#After dropping variables with p_value >=0.05, checking extracting vars on which model is amde to predic the result 
#on same vars in the test data
x_cols_in_model=list(X_variable)
x_cols_in_model.remove('const')
print('x_cols_in_model are : ',x_cols_in_model)

x_cols_in_model are :  ['x3', 'x5']


In [10]:

X_test=test_data[x_cols_in_model]
#adding constant to test data as well
X_test=sm.add_constant(X_test)

Y_test=test_data[y_var]

#Validation 
print('Train_Variable Columns sequence is : ',list(X_variable))
print('Train_Variable Columns sequence is : ',list(X_test))


pred_DepVar_test=linear_model.predict(X_test)


test_data=pd.concat((test_data,pred_DepVar_test),axis=1)
test_data=test_data.rename(columns={0:'pred_DepVar'})
test_data['Sample']='Test'
test_data.info()
test_data.head()

Train_Variable Columns sequence is :  ['const', 'x3', 'x5']
Train_Variable Columns sequence is :  ['const', 'x3', 'x5']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DepVar       21 non-null     int64  
 1   x1           21 non-null     float64
 2   x2           21 non-null     float64
 3   x3           21 non-null     int64  
 4   x4           21 non-null     float64
 5   x5           21 non-null     float64
 6   x6           21 non-null     int64  
 7   x7           21 non-null     float64
 8   x8           21 non-null     float64
 9   x9           21 non-null     float64
 10  x10          21 non-null     int64  
 11  x11          21 non-null     float64
 12  x12          21 non-null     float64
 13  x13          21 non-null     float64
 14  x14          21 non-null     int64  
 15  pred_DepVar  21 non-null     float64
 16  Sample       21 

,DepVar,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,pred_DepVar,Sample
0,29900077,0.0,0.0,1,1.0,0.0,0,0.0,0.0,0.0,0,1.0,1.0,0.0,0,5.018859e+07,Test
1,67020000,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,5.042671e+07,Test
2,93540090,0.0,0.0,0,0.0,0.0,0,8.0,8.0,0.0,0,2.0,2.0,0.0,0,5.042671e+07,Test
3,70390135,0.0,0.0,1,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,5.018859e+07,Test
4,13416145,1.0,0.0,3,4.0,0.0,0,2.0,2.0,0.0,0,0.0,0.0,0.0,0,4.971234e+07,Test


### Combining Train and test Scoring

In [11]:
final_data=pd.concat([train_data,test_data],axis=0)
final_data=final_data.reset_index()
final_data.drop('index',axis=1,inplace=True)
final_data.info()
final_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DepVar       695 non-null    int64  
 1   x1           695 non-null    float64
 2   x2           695 non-null    float64
 3   x3           695 non-null    int64  
 4   x4           695 non-null    float64
 5   x5           695 non-null    float64
 6   x6           695 non-null    int64  
 7   x7           695 non-null    float64
 8   x8           695 non-null    float64
 9   x9           695 non-null    float64
 10  x10          695 non-null    int64  
 11  x11          695 non-null    float64
 12  x12          695 non-null    float64
 13  x13          695 non-null    float64
 14  x14          695 non-null    int64  
 15  pred_DepVar  695 non-null    float64
 16  Sample       695 non-null    object 
dtypes: float64(11), int64(5), object(1)
memory usage: 92.4+ KB


,DepVar,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,pred_DepVar,Sample
0,15090000,1.92,0.0,0,1.92,16.32,0,0.0,16.32,0.0,0,6.0,6.0,0.0,0,3.945436e+07,Train
1,13405233,0.00,0.0,4,4.00,0.00,2,5.0,7.00,0.0,1,2.0,3.0,0.0,0,4.947422e+07,Train
2,70390700,0.00,0.0,0,0.00,0.00,25,0.0,25.00,0.0,4,0.0,4.0,0.0,4,5.042671e+07,Train
3,80030330,0.00,1.0,0,1.00,0.00,1,0.0,1.00,0.0,1,0.0,1.0,0.0,1,5.042671e+07,Train
4,89201260,0.00,0.0,0,0.00,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0,5.042671e+07,Train


### Coeff and p values

In [12]:
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 DepVar   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     6.350
Date:                Sun, 14 Jan 2024   Prob (F-statistic):            0.00185
Time:                        18:55:39   Log-Likelihood:                -12550.
No. Observations:                 674   AIC:                         2.511e+04
Df Residuals:                     671   BIC:                         2.512e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.043e+07   1.19e+06     42.494      0.000    4.81e+07    5.28e+07
x3         -2.381e+05   1.09e+05     -2.192      0.029   -4.51e+05   -2.48e+04
x5         -6.723e+05   2.43e+05     -2.761      0.006   -1.15e+06   -1.94e+05
==============================================================================
Omnibus:                     1584.306   Durbin-Watson:                   1.432
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.475
Skew:                           0.079   Prob(JB):                     1.81e-11
Kurtosis:                       1.682   Cond. No.                         11.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
#pvalues
pvalues=pd.DataFrame(linear_model.pvalues)
pvalues.rename(columns={0:'p_values'},inplace=True)
pvalues

,p_values
const,1.881108e-192
x3,2.872359e-02
x5,5.916897e-03


In [14]:
#beta coeff
beta_values=pd.DataFrame(linear_model.params)
beta_values.rename(columns={0:'beta_coefficients'},inplace=True)
beta_values

,beta_coefficients
const,5.042671e+07
x3,-2.381225e+05
x5,-6.723249e+05


### Exporting Scoring, pvalues, beta co-eff in excel 

In [16]:
with pd.ExcelWriter('OLS_Output.xlsx', engine='xlsxwriter') as writer:
    final_data.to_excel(writer, sheet_name='Train_Test_Scoring')
    pvalues.to_excel(writer, sheet_name='pvalues')
    beta_values.to_excel(writer, sheet_name='beta_coefficients')